<img src="https://github.com/hernancontigiani/ceia_memorias_especializacion/raw/master/Figures/logoFIUBA.jpg" width="500" align="center">


# Procesamiento de lenguaje natural
## Modelo de lenguaje con tokenización por caracteres

### Consigna
- Seleccionar un corpus de texto sobre el cual entrenar el modelo de lenguaje.
- Realizar el pre-procesamiento adecuado para tokenizar el corpus, estructurar el dataset y separar entre datos de entrenamiento y validación.
- Proponer arquitecturas de redes neuronales basadas en unidades recurrentes para implementar un modelo de lenguaje.
- Con el o los modelos que consideren adecuados, generar nuevas secuencias a partir de secuencias de contexto con las estrategias de greedy search y beam search determístico y estocástico. En este último caso observar el efecto de la temperatura en la generación de secuencias.


### Sugerencias
- Durante el entrenamiento, guiarse por el descenso de la perplejidad en los datos de validación para finalizar el entrenamiento. Para ello se provee un callback.
- Explorar utilizar SimpleRNN (celda de Elman), LSTM y GRU.
- rmsprop es el optimizador recomendado para la buena convergencia. No obstante se pueden explorar otros.


### Imports

In [3]:
import numpy as np
import urllib.request
import bs4 as bs

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.optimizers import Adam

### Corpus

In [4]:
url = "https://www.textos.info/miguel-de-cervantes-saavedra/el-ingenioso-hidalgo-don-quijote-de-la-mancha/ebook"

raw_html = urllib.request.urlopen(url).read()

article_html = bs.BeautifulSoup(raw_html, "lxml")

article_paragraphs = article_html.find_all("p")

article_text = " ".join(p.text for p in article_paragraphs)

article_text = article_text.lower()

print("Cantidad de caracteres en el corpus (antes de recorte):", len(article_text))
print(article_text[:1000])


Cantidad de caracteres en el corpus (antes de recorte): 2077830
 yo, juan gallo de andrada, escribano de cámara del rey nuestro señor, de
los que residen en su consejo, certifico y doy fe que, habiendo visto por
los señores dél un libro intitulado el ingenioso hidalgo de la mancha,
compuesto por miguel de cervantes saavedra, tasaron cada pliego del dicho
libro a tres maravedís y medio; el cual tiene ochenta y tres pliegos, que
al dicho precio monta el dicho libro docientos y noventa maravedís y medio,
en que se ha de vender en papel; y dieron licencia para que a este precio
se pueda vender, y mandaron que esta tasa se ponga al principio del dicho
libro, y no se pueda vender sin ella. y, para que dello conste, di la
presente en valladolid, a veinte días del mes de deciembre de mil y
seiscientos y cuatro años. juan gallo de andrada. este libro no tiene cosa digna que no corresponda a su original; en
testimonio de lo haber correcto, di esta fee. en el colegio de la madre de
dios de los te

Utilicé como corpus *“Don Quijote de la Mancha”* en formato ebook desde textos.info. Después de descargar y parsear el HTML, uní todos los párrafos en un único string y convertí el texto a minúsculas. El resultado es un texto largo en español, suficiente para que el modelo aprenda patrones básicos de ortografía, puntuación y estilo.

### Tokenización de caracteres con Keras

In [5]:
# Tokenizador a nivel carácter
tokenizer = Tokenizer(
    char_level=True,
    filters="",
    lower=False,
    oov_token=None
)

tokenizer.fit_on_texts([article_text])

char2idx = tokenizer.word_index.copy()
idx2char = {idx: ch for ch, idx in char2idx.items()}

vocab_size = len(char2idx) + 1

print("Tamaño vocabulario (nº de caracteres diferentes):", vocab_size)
print("Algunos pares char -> idx:", list(char2idx.items())[:30])


Tamaño vocabulario (nº de caracteres diferentes): 66
Algunos pares char -> idx: [(' ', 1), ('e', 2), ('a', 3), ('o', 4), ('s', 5), ('n', 6), ('r', 7), ('l', 8), ('d', 9), ('u', 10), ('i', 11), ('t', 12), ('c', 13), ('m', 14), (',', 15), ('p', 16), ('q', 17), ('\n', 18), ('y', 19), ('b', 20), ('h', 21), ('v', 22), ('g', 23), ('í', 24), ('j', 25), ('ó', 26), ('.', 27), ('f', 28), ('é', 29), ('á', 30)]


### Texto codificado y construcción del dataset

In [6]:
encoded = tokenizer.texts_to_sequences([article_text])[0]
print("Largo de la secuencia codificada:", len(encoded))

seq_length = 100

sequences = []

for i in range(seq_length, len(encoded)):
    seq = encoded[i - seq_length : i + 1]
    sequences.append(seq)

sequences = np.array(sequences)
print("Shape de sequences (total_secuencias, seq_length+1):", sequences.shape)

X = sequences[:, :-1]
y = sequences[:, -1]

print("Shape X:", X.shape)
print("Shape y:", y.shape)

split_idx = int(len(X) * 0.9)

X_train, X_val = X[:split_idx], X[split_idx:]
y_train, y_val = y[:split_idx], y[split_idx:]

X_train = X_train.astype("int32")
X_val   = X_val.astype("int32")
y_train = y_train.astype("int32")
y_val   = y_val.astype("int32")

print("X_train:", X_train.shape, " | y_train:", y_train.shape)
print("X_val:  ", X_val.shape,   " | y_val:  ", y_val.shape)


Largo de la secuencia codificada: 2077830
Shape de sequences (total_secuencias, seq_length+1): (2077730, 101)
Shape X: (2077730, 100)
Shape y: (2077730,)
X_train: (1869957, 100)  | y_train: (1869957,)
X_val:   (207773, 100)  | y_val:   (207773,)


Apliqué una tokenización a nivel carácter usando `Tokenizer` de Keras, asignando un índice entero a cada símbolo distinto del texto. Con la secuencia codificada construí ventanas de longitud fija (`seq_length = 100`), donde cada muestra usa 100 caracteres como entrada y el carácter siguiente como etiqueta. Finalmente separé el conjunto en entrenamiento y validación (90% / 10%) para evaluar la capacidad de generalización del modelo.

### Modelo Keras (Embedding + LSTM)

In [7]:
embedding_dim = 64
rnn_units = 128

model = Sequential([
    Embedding(
        input_dim=vocab_size,
        output_dim=embedding_dim,
        input_length=seq_length
    ),
    LSTM(rnn_units, return_sequences=False),
    Dense(vocab_size, activation="softmax")
])

model.compile(
    loss="sparse_categorical_crossentropy",
    optimizer=Adam(learning_rate=0.001)
)

model.summary()


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

### Entrenamiento del modelo

In [8]:
EPOCHS = 10
BATCH_SIZE = 128

history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=EPOCHS,
    batch_size=BATCH_SIZE
)

Epoch 1/10
14610/14610 ━━━━━━━━━━━━━━━━━━━━ 136s 9ms/step - loss: 2.0024 - val_loss: 1.5932
Epoch 2/10
14610/14610 ━━━━━━━━━━━━━━━━━━━━ 133s 9ms/step - loss: 1.5442 - val_loss: 1.5002
Epoch 3/10
14610/14610 ━━━━━━━━━━━━━━━━━━━━ 133s 9ms/step - loss: 1.4591 - val_loss: 1.4570
Epoch 4/10
14610/14610 ━━━━━━━━━━━━━━━━━━━━ 133s 9ms/step - loss: 1.4184 - val_loss: 1.4319
Epoch 5/10
14610/14610 ━━━━━━━━━━━━━━━━━━━━ 133s 9ms/step - loss: 1.3941 - val_loss: 1.4158
Epoch 6/10
14610/14610 ━━━━━━━━━━━━━━━━━━━━ 133s 9ms/step - loss: 1.3763 - val_loss: 1.4056
Epoch 7/10
14610/14610 ━━━━━━━━━━━━━━━━━━━━ 133s 9ms/step - loss: 1.3632 - val_loss: 1.4006
Epoch 8/10
14610/14610 ━━━━━━━━━━━━━━━━━━━━ 134s 9ms/step - loss: 1.3548 - val_loss: 1.3932
Epoch 9/10
14610/14610 ━━━━━━━━━━━━━━━━━━━━ 140s 10ms/step - loss: 1.3465 - val_loss: 1.3870
Epoch 10/10
14610/14610 ━━━━━━━━━━━━━━━━━━━━ 134s 9ms/step - loss: 1.3370 - val_loss: 1.3824



El modelo está formado por una capa `Embedding`, una capa `LSTM` con 128 unidades y una capa densa final con activación `softmax` sobre todo el vocabulario de caracteres. Lo entrené con `sparse_categorical_crossentropy` y el optimizador `Adam`, usando batches intermedios y varias épocas. Durante el entrenamiento la pérdida de entrenamiento y validación disminuye de forma estable, lo que muestra que el modelo va capturando patrones del corpus sin evidenciar un sobreajuste fuerte.

### Función para generar texto

In [9]:
def sample_with_temperature(preds, temperature=1.0):
    preds = np.asarray(preds).astype("float64")
    preds = np.log(preds + 1e-8) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    return np.random.choice(len(preds), p=preds)

def generate_text(seed_text, gen_length=500, temperature=1.0):
    text = seed_text.lower()

    for _ in range(gen_length):
        encoded_seed = tokenizer.texts_to_sequences([text[-seq_length:]])[0]

        if len(encoded_seed) < seq_length:
            encoded_seed = [0] * (seq_length - len(encoded_seed)) + encoded_seed
        else:
            encoded_seed = encoded_seed[-seq_length:]

        x_pred = np.array(encoded_seed)[None, :]

        preds = model.predict(x_pred, verbose=0)[0]
        next_index = sample_with_temperature(preds, temperature=temperature)
        next_char = idx2char.get(next_index, "")

        text += next_char

    return text


### Test de generación de texto

In [10]:
seed = "en un lugar de la mancha"
generated = generate_text(seed_text=seed, gen_length=500, temperature=0.8)

print(generated)

en un lugar de la mancha, si
no se diese sin hacer, estos
remosas que respondió el aina y ha talla que llegan satisfación en la hija; los cuales de aquella mayor corresponete. pero no hay todos: esto había
oído mandatarle la legua que con pedras; porque corre que llevase ellos no fueran volver en los digos, en priesco del mundo el todo la boca que se pienso, las hallado mujer a mismo ella caballero, hecho ser demalo con las
mejores cañías de los aconsejos, y le
ha de todos aquellos a la tierra. y todo estará
así lo que


Para evaluar el modelo generé texto a partir de una frase semilla, por ejemplo “en un lugar de la mancha”. El modelo produce secuencias en español con rasgos del estilo del corpus, aunque todavía aparecen palabras inventadas y la coherencia se pierde en textos más largos. La temperatura de muestreo permite ajustar el equilibrio entre diversidad y estabilidad, valores bajos en la temperatura generan texto más predecible y repetitivo, mientras que valores altos producen resultados más creativos pero también más ruidosos.